In [ ]:
from pydub import AudioSegment
from scipy import signal
import torch
import torchvision
import torchvision.transforms as transforms
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import matplotlib.pyplot as plt
import numpy as np
from pydub import playback
from pydub import generators
import array
import time
import musdb
from torch.utils import data

class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(2,16,3,padding=1)
        self.conv2 = nn.Conv2d(16,32,3,padding=1)
        self.conv3 = nn.Conv2d(32,64,5)
        self.pool1 = nn.MaxPool2d((5,2))
        self.conv4 = nn.Conv2d(64,96,3,padding=1)
        self.conv5 = nn.Conv2d(96,128,3,padding=1)
        self.conv6 = nn.Conv2d(128,160,3,padding=1)
        self.pool2 = nn.MaxPool2d((2,2))
        self.conv7 = nn.Conv2d(160,192,3,padding=1)
        self.pool3 = nn.MaxPool2d((2,2))
        self.conv8 = nn.Conv2d(192,224,3,padding=1)
        self.pool4 = nn.MaxPool2d((5,2))
        self.conv9 = nn.Conv2d(224,256,3,padding=1)
        self.upsamp1 = nn.Upsample(size=(51,10),mode = 'bilinear')
        self.conv10 = nn.Conv2d(480,256,3,padding=1)
        self.conv11 = nn.Conv2d(256,128,3,padding=1)
        self.conv12 = nn.Conv2d(128,80,3,padding=1)
        self.upsamp2 = nn.Upsample(size=(102,21),mode = 'bilinear')
        self.conv13 = nn.Conv2d(272,128,3,padding=1)
        self.conv14 = nn.Conv2d(128,96,3,padding=1)
        self.conv15 = nn.Conv2d(96,80,3,padding=1)
        self.upsamp3 = nn.Upsample(size=(204,42),mode = 'bilinear')
        self.conv16 = nn.Conv2d(240,128,3,padding=1)
        self.conv17 = nn.Conv2d(128,96,3,padding=1)
        self.conv18 = nn.Conv2d(96,80,3,padding=1)
        self.upsamp4 = nn.Upsample(size=(1025,88),mode = 'bilinear')
        self.conv19 = nn.Conv2d(80,48,3,padding=1)
        self.conv20 = nn.Conv2d(48,32,3,padding=1)
        self.conv21 = nn.Conv2d(32,12,3,padding=1)
        self.conv22 = nn.Conv2d(12,6,3,padding=1)
        
    def forward(self, x):   
        x = F.relu(self.conv1(x))
        x = F.relu(self.conv2(x))
        x = self.pool1(F.relu(self.conv3(x)))
        x = F.relu(self.conv4(x))
        x = F.relu(self.conv5(x))
        x_1 = self.conv6(x)
        x = self.pool2(F.relu(x_1))
        x_2 = self.conv7(x)
        x = self.pool3(F.relu(x_2))
        x_3 = self.conv8(x)
        x = self.pool4(F.relu(x_3))
        x = self.conv9(x)
        x = self.upsamp1(x)
        x_prime = torch.cat((x,x_3),1)
        x = F.relu(self.conv10(x_prime))
        x = F.relu(self.conv11(x))
        x = F.relu(self.conv12(x))
        x = self.upsamp2(x)
        x_prime = torch.cat((x,x_2),1)
        x = F.relu(self.conv13(x_prime))
        x = F.relu(self.conv14(x))
        x = F.relu(self.conv15(x))
        x = self.upsamp3(x)
        x_prime = torch.cat((x,x_1),1)
        x = F.relu(self.conv16(x_prime))
        x = F.relu(self.conv17(x))
        x = F.relu(self.conv18(x))
        x = self.upsamp4(x)  
        x = F.relu(self.conv19(x))
        x = F.relu(self.conv20(x))
        x = F.relu(self.conv21(x))
        x = self.conv22(x)
        
        return x